In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

## Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'Emma'
rep1_2_path = base_path/'2024.06.05_EXP56'/'data_singlets'
rep3_path = base_path/'2024.06.09_EXP60'/'data_singlets'

plates = pd.DataFrame({
    'data_path': [rep1_2_path, rep1_2_path, rep3_path],
    'yaml_path': [rep1_2_path/'elp_exp56_biorep_1_wells.yaml', rep1_2_path/'elp_exp56_biorep_2_wells.yaml', rep3_path/'elp_exp56_biorep_3_wells.yaml'],
    'exp': ['elp_exp56.1','elp_exp56.2','elp_exp60'],
    'biorep': [1,2,3]
})

output_path = rd.rootdir/'output'/'therapeutic-genes-transfection'
cache_path = output_path/'data.gzip'

metadata_keys = set()
for p in plates['yaml_path'].unique():
    print(p)
    rd.plot.plot_well_metadata(p)
    metadata_keys.update(rd.flow.load_well_metadata(p).keys())
display(metadata_keys)

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mRuby2-A','EGFP-A','iRFP-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')

# Define colors
metadata.loc[metadata['ts_kind']=='na', 'color'] = 'black'
metadata.loc[metadata['ts_kind']=='NT', 'color'] = base.colors['gray']
metadata.loc[metadata['ts_kind']=='T', 'color'] = base.colors['teal']

# Define markers
metadata.loc[metadata['group']=='base', 'markers'] = 'X'
metadata.loc[metadata['group']=='controller', 'markers'] = 'o'

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color']
construct_markers = metadata_dict['markers']

data = data.merge(metadata, how='left', on='construct')

In [ ]:
gates = pd.DataFrame()
channel_list = ['iRFP-A', 'mRuby2-A', 'EGFP-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Indicate which channels are relevant for each construct
gates['marker'] = 'iRFP-A'

display(gates)

In [ ]:
def gate_data(df, gates):
    df = df.copy()
    exp = df['exp'].values[0] # the same for entire df, assuming df = data.groupby('exp')
    gates_dict = gates.set_index('exp').to_dict('dict') # format: column -> {index: value}
    marker = gates_dict['marker'][exp]
    df['expressing'] = df[marker] > gates_dict[marker][exp]
    df['marker'] = df[marker]
    return df

data = data.groupby(['exp','construct'])[data.columns].apply(lambda x: gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
df = data[(data['expressing']) & (data['construct']!='UT')]
display(df)

In [ ]:
# Add output column based on construct
df['gene'] = 'FP'
df.loc[df['construct'].isin(['RC256','RC257','RC258']), 'gene'] = 'FXN'
df.loc[df['construct'].isin(['RC262','RC263','RC264']), 'gene'] = 'FMRP'

df['output'] = df['mRuby2-A']
df.loc[df['gene']=='FMRP', 'output'] = df.loc[df['gene']=='FMRP', 'EGFP-A']
display(df)

In [ ]:
# Bin by marker quantiles
by = ['construct','exp','biorep']
df['bin_marker_quantiles'] = df.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles = df.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])
display(df_quantiles)

# Quantile stats & slope
df_quantiles['bin_marker_quantiles_median_log'] = df_quantiles['bin_marker_quantiles_median'].apply(np.log10)
df_quantiles['output_log'] = df_quantiles['output'].apply(np.log10)

stat_list = [sp.stats.gmean, ]#sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=by+['bin_marker_quantiles_median'])
stats_quantiles = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_quantiles.columns = stats_quantiles.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_quantiles['count'] = grouped['output'].count().reset_index()['output']
stats_quantiles = stats_quantiles.merge(metadata, how='left', on='construct')
display(stats_quantiles)

stats_quantiles['bin_marker_quantiles_median_log'] = stats_quantiles['bin_marker_quantiles_median'].apply(np.log10)
stats_quantiles['output_gmean_log'] = stats_quantiles['output_gmean'].apply(np.log10)

# Compute slope for all constructs
def get_slope(df):
    slope, intercept, r_value, p_value, stderr = sp.stats.linregress(df['bin_marker_quantiles_median_log'], df['output_gmean_log'])
    result = pd.DataFrame(columns=['slope', 'intercept_log', 'r_value', 'p_value', 'stderr'])
    result.loc[len(result.index)] = [slope, intercept, r_value, p_value, stderr]
    return result

fits = stats_quantiles.groupby(by)[stats_quantiles.columns].apply(get_slope).reset_index()
fits['intercept'] = fits['intercept_log'].apply(lambda x: 10**x)
fits = fits.merge(metadata, how='left', on='construct')
display(fits)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
biorep = 2
plot_df = df_quantiles[(df_quantiles['biorep']==biorep) & (df_quantiles['gene']!='FMRP')]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=construct_palette, 
             legend=False, dashes=False, style='construct', markers=construct_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_FXN.svg')), bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
biorep = 2
plot_df = df_quantiles[(df_quantiles['biorep']==biorep) & (df_quantiles['gene']=='FMRP')]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=construct_palette, 
             legend=False, dashes=False, style='construct', markers=construct_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='construct', palette=construct_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_FMRP.svg')), bbox_inches='tight')